In [162]:
#calcuration libraries
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

### Experimental facilities

In [163]:
#pipe diameter[m]
d_i = 12 * 10**(-3)
d_o = 15 * 10**(-3)
#pipe area[m**2]
A = (np.pi * d_i**2) / 4
#heated length[m]
l_heated = 2
#distance of pressure mesurement[m]
dz = 1

### Material properties

In [164]:
A = 0.818;
B = 3.664E-3;

def cp(T):
    cp = (A + B * (T+273.15));
    return cp
print(cp(0))

1.8188215999999997


In [165]:
def cp_K(T):
    cp_K = A + B * (T);
    cp_K =  cp_K *1000;##???
    return cp_K
print(cp_K(273.15))

1818.8215999999998


In [166]:
C = 0.157;
D = -7.328E-5;

def Lambda(T):
    Lambda = C + D * (T+273.15);
    return Lambda
print(Lambda(0))

0.136983568


In [167]:
def Lambda_K(T):
    Lambda_K = C + D * (T);
    return Lambda_K
print(Lambda_K(273.15))

0.136983568


In [168]:
E = 5.894E-5;
F = 857.4;
G = -172.2;

def mu(T):
    mu = E * np.exp( F / ( (T+273.15) + G) );
    ##original mu = A .* exp( B ./ ( (T) + C) );
    return mu
print(mu(0))

0.2877460210984148


In [169]:
def mu_K(T):
    mu_K = E * np.exp( F / ( (T) + G) );
    return mu_K
print(mu_K(273.15))

0.2877460210984148


In [170]:
H = 1045;
I = -0.616;
def rho(T):
    rho = H + I * (T+273.15);
    return rho
print(rho(0))

876.7396


In [171]:
def rho_K(T):
    rho_K = H + I * (T);
    return rho_K
print(rho_K(273.15))

876.7396


In [172]:
def nu(T):
    nu = mu(T)/rho(T);
    return nu
print(nu(0))

0.0003282000962411357


In [173]:
##nuとの違い？
def nu_K(T):
    nu = mu_K(T)/rho_K(T);
    return nu
print(nu(0))

0.0003282000962411357


In [174]:
def Pr(T):
    Pr = ( mu(T) * cp_T(T) * 1000 ) / lambda_T(T);
    return Pr
print(Pr(0))

3820.5945875774855


In [175]:
##Prとの違い？
def Pr_K(T):
    Pr = ( mu_K(T) * cp_K(T)) / lambda_K(T);
    return Pr
print(Pr(273))

14.066109332977256


### read measurement data

In [176]:
# 取り出したい列を（0始まりで）指定する．
import pandas as pd
filename = './re230pr45/Daten_19.11.2018_16.02.21.csv'
data = np.array(pd.read_csv(filename, skiprows=0, encoding='shift-jis'))

In [177]:
di=12E-3;
da=15E-3;
disoa = 90E-3;
L=2;
V=(da**2-di**2)*np.pi/4*L;
Ageo_MS = di*np.pi*L;#円菅内部の試験部表面積
Ugeo_MS = di*np.pi;#円菅内面積
Ageo_quer = di**2*np.pi/4;

Tsa = data[:,0];#Temperatur aussen: Ts_aussen / °C
Tsi = data[:,1];#Temperatur innen: Ts innen / °C
Ti = data[:,2];#Fluidtemperatur: T / °C
m_dot_C1 = data[0,3];#Massenstrom: m_dot C1 / kg/h
m_dot_C2 = data[0,4];#Massenstrom: m_dot C2 / kg/h

Re_C1 = data[0,5];#Re C1 / -
Re_C2 = data[0,6];#Re C2 / -

v_C1 = data[0,7];#Geschwindigkeit: v C1 / m/s
v_C2 = data[0,8];#Geschwindigkeit: v C2 / m/s

U_MS = data[0,9];#Spannungsabfall Messstrecke: U_MS / V
I_MS = data[0,10];#Strom Messstrecke: I_MS / A
P_MS = data[0,11];#Leistung Messstrecke: P_MS / W

q = data[0,12];#spzifische Leistung: q / W/m3
P1 = data[0,13];#Druckmessung: P1 / Pa
P2 = data[0,14];#Druckmessung: P2 / Pa
dp = data[0,15];#Differenzdruck: dp / bar
nu_Fluid = data[0,16];#kin. Viscositaet Fluid: nu / m2/s
rho_Fluid = data[0,17];#Dichte Fluid: rho / kg/m3
lambda_Fluid = data[0,18];#Waermeleitfaehigkeit Fluid: lambda / W/mK
cp_Fluid = data[0,19];#spez. Waermekapazitaet Fluid: cp /J/kgK
Pr_Fluid = data[0,20];#Pr / -
lambda_Rohr = data[0,21];#Waermeleitfaehigkeit Rohr: lambda / W/mK
Nu_Fluid = data[0,22];#Nu / -

I1 = data[0,23];#Strom I / A
U1 = data[0,24];#Spannung U / V

dp_T1 = data[0,25];#Re_tau / - 関数名とtxtが合っていない？
dp_T2 = data[0,26];#Re_tau_Pet / -　関数名とtxtが合っていない？
dp_T3 = data[0,27];#dp Pet / Pa

#Mittel (U*I) mean P / W
Pm = data[0,28];#Mittel (U*I) mean P / W
TPT100_a = data[:,29];#PT100 T aussen(5 value)
TPT100_i = data[:,30];#PT100 T innen(5 value)
TPT100_m = data[:,31];#PT100 T mittel(5 value)

NuPt100 = data[0,32];#Nu Pt100
NuQm = data[0,33];#Nu Qm

# zeta computation Einlesen
ZRem100 = data[0,34];#zeta Re
Zzeta100 = data[0,35];#zeta
Zqm100 = data[0,36];#zeta qw

TPT100_T5 = data[0,37];#PT100 T5

print(Ugeo_MS)

0.03769911184307752


In [178]:
# T_aussen -> T_innen (check)
lambda_K = 0.16;
ri       = di/2;
ra       = da/2;
DeltaK   = 0.065E-3;
#lambda_N = @(T) 20;%20;%0.0135*T + 15.1622;%14;
#conduction equation for inner wall temperatuer

### Definition der Temperaturen und MesspositionenCoeff_lf

In [179]:
Tsa_up = [Tsa[0], Tsa[2], Tsa[4], Tsa[6], Tsa[8], Tsa[10], Tsa[12], Tsa[14], Tsa[16], Tsa[18], Tsa[20], Tsa[22]]
Tsa_down = [Tsa[1], Tsa[3], Tsa[5], Tsa[7], Tsa[9], Tsa[11], Tsa[13], Tsa[15], Tsa[17], Tsa[19], Tsa[21], Tsa[23]]
Tsi_up = [Tsi[0], Tsi[2], Tsi[4], Tsi[6], Tsi[8], Tsi[10], Tsi[12], Tsi[14], Tsi[16], Tsi[18], Tsi[20], Tsi[22]]
Tsi_down = [Tsi[1], Tsi[3], Tsi[5], Tsi[7], Tsi[9], Tsi[11], Tsi[13], Tsi[15], Tsi[17], Tsi[19], Tsi[21], Tsi[23]]

x_pos_Ts_up = [1.230, 1.520, 2.020, 2.420, 2.720, 2.870, 3.020, 3.170, 3.230, 4.680, 4.720, 5.480, 5.520]
x_pos_Ts_up_a = [1.230, 1.520, 2.020, 2.420, 2.720, 2.870, 3.020, 3.170, 3.230, 4.680, 4.720, 5.480, 5.520]
x_pos_Ts_down = [1.230, 1.520, 2.020, 2.420, 2.870, 3.020, 3.170, 3.230, 5.480, 5.520]
x_pos_Ts_down_a = [1.230, 1.520, 2.020, 2.420, 2.870, 3.020, 3.170]
x_pos_t=[2.720, 4.700]

T_Ms_fluid = [Ti[2], Ti[0]]
x_pos_p = [3.3, 4.3, 4.4]
pos = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]#なんとかならない？

x_pos_TPt100 = [2.920, 0.05, 2.720, 5.500]
x_pos_TPt100 = [2.420, 0.05, 2.920, 5.500]
x_pos_TPt100 = [1.970, 2.470, 2.970, 5.500, 0.05]
xTin  = 0.05;
xTout = 5.50;
#x_pos_TPt100 = [2.720 3.170 3.170 4.700];

x_MS_in = 1.2;
x_MS_out = 3.2;

In [180]:
#TPT100_a = TPT100_mean
TPT100 = np.zeros((1,5))
TPT100 = TPT100_i
print(TPT100)#TPT100の5個目以降の値をゼロにしたい！！！！！！！！！！！！！！

Tm_aus=TPT100[3];
Tm=Tm_aus;
print(Tm)#オフセットあるので違う！！！！！！！！！！！！！

[123.672559 125.84253  126.312521  98.763218  95.438691   0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.      ]
98.76321800000001


### iteration to update the temperatuer dependent material properties --> new wall temperature

In [181]:
Tw = TPT100[2]
print(Tw)#オフセットあるので違う！！！！！！！！！！！！！

T_iso = Ti[1]#断熱材の外の温度？
T_inf = Ti[6]#Temperature infinity

126.31252099999999


### Berechnung der Fluid Stoffwerte

In [182]:
Pr_w = Pr(Tw)

Pr_w     = Pr(Tw)
rho_w    = rho(Tw)
nu_w     = nu(Tw)
mu_w     = nu_w * rho_w
cp_w     = cp(Tw)
lambda_w = Lambda(Tw)

Pr_m     = Pr(Tm)
rho_m    = rho(Tm)
nu_m     = nu(Tm)
mu_m     = nu_m * rho_m
cp_m     = cp(Tm)
lambda_m = Lambda(Tm)

Re_C1;

### Korrektur mittel Temperatur

In [183]:
#dqhldT = 1.;
#zeta_mischer = 32.46;

#Thl = (Tw+Tm)*0.5;
#qhl= dqhldT* (Thl-T_iso);

#qhlm2 = qhl / (da*pi*5.45);

Re_m = Re_C1;
lambda_Pet = (1.8*np.log(Re_m)-1.5)**(-2.);

In [184]:
#Specific heat capasity, cp mean
T_in=TPT100[4]+273.15;
T_out=TPT100[3]+273.15;

Ac = 0.818;
Bc = 3.664E-3;
cpm=(Ac*(T_out-T_in)+Bc/2*(T_out**2-T_in**2))/(T_out-T_in)*1000;#Eq(2.108)

Wmean=Re_m*nu_m/di;
mdot = m_dot_C1/3600;

In [185]:
#Q= DT_pt100 / Dx
    
#fit=polyfit(x_pos_TPt100(1:3),TPT100(1:3)',1);
#gradT_PT100=fit(1);

#Q_Pt100 = gradT_PT100*mdot*cpm*L;

#qUI=P_MS/Ageo_MS;

#dqw2 = (lambda_Pet*(5.45-2.92)/di + zeta_mischer) * rho_m * Wmean^2/2 * Wmean * Ageo_quer - qhl*(5.45-2.53)/5.45;

#Qin_out=max(mdot*cpm*(T_out-T_in)-(lambda_Pet*(5.45-2.92)/di + zeta_mischer) * rho_m * Wmean^2/2 * Wmean * Ageo_quer + qhl,0);
Qin_out=mdot*cpm*(T_out-T_in);
dQ=Qin_out-P_MS;

In [186]:
#Q=m_dot * cp_m * DT;
#gradT = (Qin_out/L) / ( (mdot) * cpm );

#dTm = dqw2/(mdot*cpm);

#SA_Tm=gradT*(x_MS_out-x_pos_TPt100(3));

In [187]:
#Tm_aus = TPT100(4) - dTm;
#Tm = Tm_aus - SA_Tm;

qw = Qin_out/ Ageo_MS;#heat flux
#if(Qin_out>0)
    #qvA=qhlm2;
#else
    #qvA=0;
#end


#Pr_w     = calc_Pr(Tw);
#rho_w    = calc_rho(Tw);
#nu_w     = calc_nu(Tw);
#mu_w     = nu_w * rho_w;
#cp_w     = calc_cp(Tw);
#lambda_w = calc_lambda(Tw);

#Pr_m     = calc_Pr(Tm);
#rho_m    = calc_rho(Tm);
#nu_m     = calc_nu(Tm);
#mu_m     = nu_m * rho_m;
#cp_m     = calc_cp(Tm);
#lambda_m = calc_lambda(Tm);

In [188]:
#P_spez_m2 = P_MS/A;
#P=Pm;
#P=P_MS;

#P_spez_m2 = P/Ageo_MS;
#P_spez_m3 = P/V;

### Berechnung von Re

In [189]:
Re_m = Re_C1;
Re_w = Re_C2;
#check why there is a differnece!!!!
Re_m = m_dot_C1/3600*4/(nu_m*rho_m*di*np.pi);
Re_w = m_dot_C1/3600*4/(nu_w*rho_m*di*np.pi);

### Korrektur Tw @ Ende Messtrecke

### Nusselt

In [190]:
#Measurement
Nu_ideal =  qUI* di / (lambda_m * ( Tw-Tm ) );
Nu_QmdotCpDt_m = qw * di / (lambda_m * ( Tw-Tm ) );
Nu_QmdotCpDt_w = qw * di / (lambda_w * ( Tw-Tm ) );
qw_mdotCpDt = qw;

#Nu_fit   = gradTfit   *mdot*cpm/(di*pi) * di / ( lambda_m * ( Tw-(Tm_aus - gradTfit   *(x_MS_out-x_pos_TPt100(3)) ) ) );
#qw_fit = gradTfit   *mdot*cpm/(di*pi);
#Nu_Pt100 = gradT_PT100*mdot*cpm/(di*pi) * di / ( lambda_m * ( Tw-(Tm_aus - gradT_PT100*(x_MS_out-x_pos_TPt100(3)) ) ) );
#qw_Pt100 = gradT_PT100*mdot*cpm/(di*pi);

#NuMessung=Nu_QmdotCpDt_w;

In [195]:
print(Nu_ideal)
print(Nu_QmdotCpDt_m)
print(Nu_QmdotCpDt_w)

36.95225915450588
30.272491716808283
30.750967783750554


In [196]:
print(Tw)

126.31252099999999


### condition

In [203]:
print(Re_m)
print(Pr_w)
print(mu_m)

2312.85779394401
45.79805388546149
0.004314237922277672


In [201]:
lambda_Fluid

0.129746